# Data Set and  Feature Extraction

In this notebook, I will explain how to load and pre-process images using PyTorch `Dataset` and `DataLoader` classes. Then, I will extract encoded features for each image using CNNs. . 

Outline of this notebook:
- [Step 1](#step1): Writing custom PyTorch Dataset
- [Step 2](#step2): Using the Data Loader to obtain Batches
- [Step 3](#step3): Extracting features of all dataset images using CNN Encoder

<a id='step1'></a>
## Step 1: Writing Custom PyTorch Dataset

I wrote a custom PyTorch [Dataset](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) to recursively load all images in a directory with their paths. This dataset is an instance of my custom `ImagesDataset` class in **images_dataset.py**.  If you are unfamiliar with data loaders and datasets, you are encouraged to review [my post](http://www.sefidian.com/2022/03/09/writing-custom-datasets-and-dataloader-in-pytorch/) or [this PyTorch tutorial](http://pytorch.org/tutorials/beginner/data_loading_tutorial.html).

#### Exploring the `__getitem__` Method

The `__getitem__` method in the `ImagesDataset` class determines how an image-path pair is pre-processed before being incorporated into a batch.  This is true for all `Dataset` classes in PyTorch; if this is unfamiliar to you, please review [this link](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html). 

#### Image Pre-Processing 

Image pre-processing is relatively straightforward (from the `__getitem__` method in the `ImagesDataset` class):
```python
# Convert image to tensor and pre-process using transform
image = Image.open(image_path).convert("RGB")
if self.transform is not None:
    image = self.transform(image)
```

The `ImagesDataset` takes as input a number of arguments that can be explored in **data_loader.py**. Take the time to explore these arguments now by opening **images_dataset.py**. 
1. **`transform`** - an [image transform](http://pytorch.org/docs/master/torchvision/transforms.html) specifying how to pre-process the images and convert them to PyTorch tensors before using them as input to the CNN encoder.  I will define transforms in `transformer` variable.
2. **`directory`** - determines the directory to search for image files.
3. **`extensions`** - image file extensions to search for within the directory. 

After loading the image in the directory, the image is pre-processed using the transform that was supplied when instantiating the data loader.  

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from images_dataset import ImagesDataset
from model import EncoderCNN
import pandas as pd
from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

In [2]:
# import configs
from configs import images_dir, batch_size, embedding_size, image_resize, extensions

In [3]:
# Define a transform to pre-process the training images.
transformer = transforms.Compose(
    [
        transforms.Resize((image_resize, image_resize)),
        # convert the PIL Image to a tensor
        transforms.ToTensor(),
        # normalize image for pre-trained model
        transforms.Normalize(
            (0.485, 0.456, 0.406),
            (0.229, 0.224, 0.225),
        ),
    ]
)

In the next code cell we define a `device` that we will use move PyTorch tensors to GPU (if CUDA is available).  Run this code cell before continuing.

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In the code cells below, I will initialize the dataset and data loader.  

In [5]:
dataset = ImagesDataset(
    directory=images_dir, extensions=extensions, transform=transformer
)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [6]:
len(dataset)

187

<a id='step2'></a>
## Step 2: Using the Data Loader to obtain Batches

The implementation of CNN encoder is in the **model.py** file. The `EncoderCNN` class takes `embedding_size` as an input argument. For this project, I incorporated a pre-trained CNN into the encoder. Specifically, I used the pre-trained ResNet-50 architecture (with the final fully-connected layer removed) to extract features from a batch of pre-processed images. The output is then flattened to a vector, before being passed through a `Linear` layer to transform the feature vector to have the same size as the word embedding.

![Encoder](assets/encoder.png)

You can amend the encoder in **model.py**, to experiment with other architectures. In particular, using a [different pre-trained model architecture](http://pytorch.org/docs/master/torchvision/models.html) could be good options. If you decide to modify the `EncoderCNN` class, save **model.py** and re-execute the code cell.

Run the code cell below to instantiate the CNN encoder in `encoder`.  

In [7]:
# Initializing the encoder
encoder = EncoderCNN(embedding_size=embedding_size)
encoder.to(device)
encoder.eval()

EncoderCNN(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64

<a id='step3'></a>
## Step 3: Extracting features of all dataset images using CNN Encoder

In this step, I will pass the pre-processed images from the batch in **Step 2** of this notebook through the encoder, and then store `features` for each image in a dataframe.

In [8]:
full_features_df = pd.DataFrame([])
for images_batch, paths in tqdm(data_loader):
    images = images_batch.to(device)
    encoder.zero_grad()

    # Passing the inputs through the CNN model
    with torch.no_grad():
        features = encoder(images)
    batch_df = pd.DataFrame(features.cpu().numpy(), index=paths)
    full_features_df = full_features_df.append(batch_df)

  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
full_features_df.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
images/photo_2022-06-16_16-17-08.jpg,0.571048,0.166351,-0.031910,0.535734,-0.323734,0.004688,-0.331498,0.577268,0.318909,-0.411481,...,0.318875,0.157316,-0.660986,0.207819,-0.401372,-0.306530,0.156024,0.055571,-0.074197,0.087003
images/photo_2022-06-16_16-17-14.jpg,0.115247,0.030326,0.061850,0.102931,-0.283531,-0.504094,-0.450766,0.345489,0.069598,-0.362402,...,0.504410,0.674240,-0.027120,0.434626,-0.454244,-0.336198,0.164132,-0.204636,-0.044397,-0.071394
images/photo_2022-06-16_16-17-15.jpg,0.066969,-0.043570,0.021569,-0.022454,-0.537078,-0.140022,-0.270604,0.610856,-0.152373,-0.449084,...,0.185609,0.619882,-0.484719,0.698935,-0.359296,-0.390298,0.132568,-0.083920,0.219586,0.023857
images/photo_2022-06-16_16-17-17.jpg,-0.157355,0.150490,-0.167689,0.307654,-0.367838,-0.452446,-0.275521,0.102545,0.454681,-0.367201,...,0.434427,0.812542,-0.248000,0.358756,-0.466094,-0.334860,0.198892,-0.370750,-0.069523,-0.436752
images/photo_2022-06-16_16-17-18.jpg,0.525468,0.095109,-0.074105,0.006551,0.023072,-0.498855,-0.586702,0.277578,-0.203757,-0.573412,...,0.436376,0.752605,-0.701708,0.028035,-0.158458,-0.496632,0.494945,0.051560,0.073812,-0.383999


In [10]:
full_features_df.shape

(187, 256)

In [11]:
full_features_df.to_pickle("features.pkl")

In the next notebook I will deduplicate images using these extracted features.